Coursera Capstone Week3 Peer Graded Assessment - Part 1 (Creation of Toronto neighbourhood dataframes using pandas with the data's imported from wikipedia page)  

In [4]:
!conda install -c conda-forge beautifulsoup4 --yes

!conda install -c conda-forge geopy --yes

!conda install -c conda-forge folium=0.5.0 --yes

print('Libraries installed!')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries installed!


In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from pandas.io.json import json_normalize
import json

import requests

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported!')

Libraries imported!


In [6]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [7]:
postalCodeList = []
boroughList = []
neighborhoodList = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n')) 

In [8]:
toronto_neighorhood = [('PostalCode', postalCodeList),
                      ('Borough', boroughList),
                      ('Neighborhood', neighborhoodList)]
toronto_df = pd.DataFrame.from_dict(dict(toronto_neighorhood))
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
toronto_df_onlyassigned = toronto_df[ toronto_df.Neighborhood != 'Not assigned'].reset_index(drop=True)
toronto_df_onlyassigned.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [10]:
toronto_df_boroughgrp = toronto_df_onlyassigned.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_boroughgrp.head()


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
for index, row in toronto_df_boroughgrp.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_boroughgrp.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
toronto_df_boroughgrp.shape

(102, 3)

The cleaned up dataframe consists of <b>102 records</b> of unique postal codes with the Neighborhood under the postalcode merged together in a single row separated by comma

<b>Coursera Capstone Week3 Peer Graded Assessment - Part 2 (Association of co-oridnates for each postal codes of the Toronto neighbourhood data created above) </b>

In [13]:
!wget -q -O "Coordinates.csv" http://cocl.us/Geospatial_data
print('Coordinates downloaded!')
lat_long = pd.read_csv('Coordinates.csv')

lat_long.head()


Coordinates downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
lat_long.rename(columns= {"Postal Code":"PostalCode"}, inplace=True)

lat_long.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
toronto_df_with_cooridnates = pd.merge(toronto_df_boroughgrp,lat_long, on='PostalCode')

toronto_df_with_cooridnates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<b>Coursera Capstone Week3 Peer Graded Assessment - Part 3 (Exploring and Clustering neighborhoods of Central Toronto) </b>

In [17]:
toronto_data = toronto_df_with_cooridnates[toronto_df_with_cooridnates['Borough']=='Central Toronto'].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


In [18]:
CLIENT_ID = 'DITEDG3SYW1L02UE50LSWIYBBXBAHIAG4IIHKLAM1U1EQGUY' # your Foursquare ID
CLIENT_SECRET = '0AAPLKWZCZAPG3WYSG0ZRNOLFT02RYGNLDGWEEOQSAFBLH4I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DITEDG3SYW1L02UE50LSWIYBBXBAHIAG4IIHKLAM1U1EQGUY
CLIENT_SECRET:0AAPLKWZCZAPG3WYSG0ZRNOLFT02RYGNLDGWEEOQSAFBLH4I


In [ ]:
toronto_data.loc[1,'PostalCode']

toronto_data['Neighborhood'].value_counts

Get the first Neighbourhood co-ordinates

In [26]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


In [29]:
radius = 500
LIMIT = 100


url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=DITEDG3SYW1L02UE50LSWIYBBXBAHIAG4IIHKLAM1U1EQGUY&client_secret=0AAPLKWZCZAPG3WYSG0ZRNOLFT02RYGNLDGWEEOQSAFBLH4I&v=20180605&ll=43.6727097,-79.40567840000001&radius=500&limit=100


In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df88de10be7b43a12adff43'},
 'response': {'headerLocation': 'The Annex',
  'headerFullLocation': 'The Annex, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 43.6772097045, 'lng': -79.39946849197054},
   'sw': {'lat': 43.668209695499996, 'lng': -79.41188830802949}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ae7bc29f964a52084ad21e3',
       'name': "Ezra's Pound",
       'location': {'address': '238 Dupont St',
        'crossStreet': 'Spadina Ave',
        'lat': 43.67515283323029,
        'lng': -79.40585846415303,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67515283323029,
          'lng': -79.40585846415303}],
        'distance': 272,
  

In [32]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Ezra's Pound,Café,43.675153,-79.405858
1,Roti Cuisine of India,Indian Restaurant,43.674618,-79.408249
2,Rose & Sons,American Restaurant,43.675668,-79.403617
3,Live Organic Food Bar,Vegetarian / Vegan Restaurant,43.675053,-79.406715
4,Big Crow,BBQ Joint,43.675896,-79.403680


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

20 venues were returned by Foursquare.


<Heading>Now we'll repeat the above process to explore the venues for all the neighborhoods in Central Toronto </Heading>

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
CentralToronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

CentralToronto_venues.head()

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Booty Camp Fitness,43.728051,-79.387853,Gym / Fitness Center
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


In [37]:
print(CentralToronto_venues.shape)
CentralToronto_venues.head()

(114, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Booty Camp Fitness,43.728051,-79.387853,Gym / Fitness Center
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


In [38]:
CentralToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,35,35,35,35,35,35
Davisville North,9,9,9,9,9,9
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",15,15,15,15,15,15
"Forest Hill North, Forest Hill West",4,4,4,4,4,4
Lawrence Park,4,4,4,4,4,4
"Moore Park, Summerhill East",3,3,3,3,3,3
North Toronto West,22,22,22,22,22,22
Roselawn,2,2,2,2,2,2
"The Annex, North Midtown, Yorkville",20,20,20,20,20,20


Let's anlayze each Neighbourhood of Central Toronto

In [39]:
# one hot encoding
CenToronto_onehot = pd.get_dummies(CentralToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
CenToronto_onehot['Neighborhood'] = CentralToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [CenToronto_onehot.columns[-1]] + list(CenToronto_onehot.columns[:-1])
CenToronto_onehot = CenToronto_onehot[fixed_columns]

CenToronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Dessert Shop,Diner,Farmers Market,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
CenToronto_grouped = CenToronto_onehot.groupby('Neighborhood').mean().reset_index()
CenToronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Dessert Shop,Diner,Farmers Market,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Hotel,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.028571,0.00,0.000000,0.000000,0.028571,0.000000,0.00,0.057143,0.000000,0.000000,0.057143,0.000000,0.028571,0.085714,0.028571,0.028571,0.000000,0.028571,0.0,0.028571,0.028571,0.057143,0.000000,0.000000,0.00,0.000000,0.028571,0.000000,0.057143,0.028571,0.00,0.000000,0.000000,0.000000,0.028571,0.028571,0.085714,0.000000,0.000000,0.000000,0.028571,0.000000,0.085714,0.028571,0.000000,0.000000,0.000000,0.000000,0.057143,0.00,0.028571,0.028571,0.00,0.00,0.000000,0.000000
1,Davisville North,0.000000,0.00,0.000000,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.0,0.000000,0.000000,0.111111,0.000000,0.000000,0.00,0.222222,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.066667,0.00,0.066667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.066667,0.066667,0.000000,0.000000,0.000000,0.066667,0.000000,0.133333,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.00,0.000000,0.000000,0.00,0.00,0.066667,0.000000
3,"Forest Hill North, Forest Hill West",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.25,0.00,0.000000,0.000000
4,Lawrence Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.00,0.00,0.000000,0.000000
5,"Moore Park, Summerhill East",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000
6,North Toronto West,0.000000,0.00,0.000000,0.000000,0.000000,0.045455,0.00,0.000000,0.045455,0.136364,0.090909,0.045455,0.000000,0.045455,0.045455,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000

In [41]:
CenToronto_grouped.shape

(9, 57)

Printing top 5 most common venues in each Neighborhood

In [42]:
num_top_venues = 5

for hood in CenToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = CenToronto_grouped[CenToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
            venue  freq
0     Pizza Place  0.09
1  Sandwich Place  0.09
2    Dessert Shop  0.09
3             Gym  0.06
4     Coffee Shop  0.06


----Davisville North----
               venue  freq
0              Hotel  0.22
1  Food & Drink Shop  0.11
2     Breakfast Spot  0.11
3               Park  0.11
4        Pizza Place  0.11


----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
                 venue  freq
0          Coffee Shop  0.13
1                  Pub  0.13
2  American Restaurant  0.07
3         Liquor Store  0.07
4          Pizza Place  0.07


----Forest Hill North, Forest Hill West----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2                 Park  0.25
3     Sushi Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
                  venue  freq
0                  Park  0.25
1              Bus Line  0.25
2           Swim School  0.25
3  Gym / Fitness Center  0.25
4   

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = CenToronto_grouped['Neighborhood']

for ind in np.arange(CenToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(CenToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Gym,Coffee Shop,Italian Restaurant,Japanese Restaurant,Diner
1,Davisville North,Hotel,Breakfast Spot,Gym,Park,Clothing Store,Pizza Place,Sandwich Place,Food & Drink Shop,Yoga Studio,Farmers Market
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,American Restaurant,Supermarket,Restaurant,Liquor Store,Light Rail Station,Sports Bar,Fried Chicken Joint,Sushi Restaurant
3,"Forest Hill North, Forest Hill West",Trail,Jewelry Store,Sushi Restaurant,Park,Yoga Studio,Dessert Shop,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant
4,Lawrence Park,Gym / Fitness Center,Swim School,Bus Line,Park,Yoga Studio,Diner,History Museum,Health & Beauty Service,Gym,Greek Restaurant


<b> Clustering of Neighborhoods</b>

In [46]:
# set number of clusters
kclusters = 5

CenToronto_grouped_clustering = CenToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CenToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 3, 2, 0, 1, 0], dtype=int32)

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

CenToronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
CenToronto_merged = CenToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

CenToronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Gym / Fitness Center,Swim School,Bus Line,Park,Yoga Studio,Diner,History Museum,Health & Beauty Service,Gym,Greek Restaurant
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Hotel,Breakfast Spot,Gym,Park,Clothing Store,Pizza Place,Sandwich Place,Food & Drink Shop,Yoga Studio,Farmers Market
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Diner,Mexican Restaurant,Park,Dessert Shop,Cosmetics Shop,Health & Beauty Service
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Café,Gym,Coffee Shop,Italian Restaurant,Japanese Restaurant,Diner
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Intersection,Restaurant,Playground,History Museum,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop,Garden


<b>Visualize the clusters in folium map<\b>

In [60]:
map_clusters = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(CenToronto_merged['Latitude'], CenToronto_merged['Longitude'], CenToronto_merged['Neighborhood'], CenToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters